# Breast Cancer Detection Model Development

### Step 1: Install Required Libraries
The required libraries for the project are installed using the `pip` package manager. These include:

- **TensorFlow**: For deep learning and neural network implementation.
- **Keras**: High-level API for building and training deep learning models.
- **NumPy**: For numerical operations and array handling.
- **Pandas**: For data manipulation and analysis.
- **Scikit-learn**: For machine learning and evaluation metrics.
- **Matplotlib**: For data visualization.
- **OpenCV**: For image processing tasks.

In [ ]:
%pip install tensorflow keras numpy pandas scikit-learn matplotlib opencv-python

### Step 2: Import Libraries
Essential libraries are imported to support data preprocessing, model creation, and visualization. 
- `EfficientNetB0` is used as the base model for transfer learning.
- `ImageDataGenerator` helps with real-time data augmentation.
- `Dense`, `Flatten`, and `Dropout` layers are used to create a custom classification head.

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import EfficientNetB0 # type: ignore
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore
from tensorflow.keras.layers import Dense, Flatten, Dropout # type: ignore
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore

### Step 3: Load and Configure the Pretrained Model

1. **EfficientNetB0 Pretrained Model**: Loaded with weights from the `ImageNet` dataset and configured to exclude the top layers for transfer learning.

2. **Freezing Layers**: All layers in the base model are frozen to prevent updating their weights during training.

3. **Custom Classification Head**: A new fully connected head is added to adapt the model for binary classification (breast cancer detection).
   - `Flatten`: Flattens the output of the base model.
   - `Dense(128)`: A dense layer with 128 neurons and ReLU activation.
   - `Dropout(0.5)`: Dropout layer to prevent overfitting.
   - `Dense(1, activation='sigmoid')`: Output layer with sigmoid activation for binary classification.
   
4. **Compilation**: The model is compiled with:
   - **Adam optimizer**: For adaptive learning rate.
   - **Binary cross-entropy loss**: For binary classification tasks.
   - **Metrics**: Accuracy and AUC (Area Under the Curve).

In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'AUC'])